# Cell Membrane Segmentation
Segmentation of fluorescently labeled cell membranes using watershed segmentation

In [1]:
import numpy as np
from skimage import io, filters, feature, segmentation
from scipy import ndimage
import matplotlib.pyplot as plt

## Generate synthetic test image

In [2]:
# Create synthetic membrane-like image
img = np.zeros((200, 200))
for i in range(10):
    x = np.random.randint(0, 200)
    y = np.random.randint(0, 200)
    radius = np.random.randint(20, 40)
    rr, cc = feature.draw.circle_perimeter(x, y, radius)
    rr = np.clip(rr, 0, 199)
    cc = np.clip(cc, 0, 199)
    img[rr, cc] = 1
img = filters.gaussian(img, sigma=1)
plt.imsave('original.png', img, cmap='gray')

AttributeError: No skimage.feature attribute draw

## Enhance membrane edges using Gaussian difference

In [ ]:
edges = filters.gaussian(img, sigma=1) - filters.gaussian(img, sigma=2)
edges = edges / edges.max()  # normalize to 0-1
plt.imsave('edges.png', (edges * 255).astype(np.uint8), cmap='gray')

## Generate seeds for watershed using distance transform and local maxima

In [ ]:
distance = ndimage.distance_transform_edt(edges < 0.2)
local_max = feature.peak_local_max(distance, indices=False, min_distance=20)
markers = ndimage.label(local_max)[0]
plt.imsave('markers.png', markers.astype(np.uint8), cmap='nipy_spectral')

## Perform watershed segmentation

In [ ]:
labels = segmentation.watershed(-distance, markers, mask=edges < 0.3)
plt.imsave('segmentation.png', labels.astype(np.uint8), cmap='nipy_spectral')
np.save('labels.npy', labels)